In [2]:
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
from processor.processor import Processor as p
from database.market import Market
from strats.speculation import Speculation
import numpy as np

In [3]:
market = Market()

In [4]:
training_year = 4
strat_class = Speculation(training_year)

In [5]:
factors = [str(x) for x in range(14)]
factors.append("d1")
factors.append("d2")
factors.append("d3")

In [6]:
included_columns=[x for x in factors]
included_columns.extend(['year',"week","ticker","y"])

In [7]:
if not strat_class.check_factors():
    print("init factors")
    strat_class.assign_factors(factors)
if not strat_class.check_included_columns():
    print("init included_columns")
    strat_class.assign_included_columns(included_columns)

init factors
init included_columns


In [8]:
strat_class.pull_included_columns()
strat_class.pull_factors()

In [9]:
market.connect()
sp5 = market.retrieve("sp500")
deactivated_tickers = market.retrieve("deactivated_tickers")
unmodeled_tickers = market.retrieve("unmodeled_tickers")
market.disconnect()

In [11]:
unmodeled_tickers

,tickers
0,STZ
1,FRC
2,HSY
3,MNST
4,SBNY
5,TDG
6,V
7,ZBH
8,ZION
9,ZTS


In [16]:
tickers = [x for x in sp5["Symbol"] if x not in list(deactivated_tickers["ticker"]) and x not in list(unmodeled_tickers["tickers"])]

In [17]:
len(tickers)

403

In [18]:
market.connect()
strat_class.db.connect()
for ticker in tqdm(tickers):
    try:
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        complete = strat_class.transform(prices,ticker)
        strat_class.db.store("data",complete[strat_class.included_columns])
    except:
        continue
strat_class.db.disconnect()
market.disconnect()

  0%|                                                                                                                                                | 0/403 [00:00<?, ?it/s]E:\projects\longshot_venv\longshot\processor\processor.py:19: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data["week"] = data["date"].dt.week
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 403/403 [00:43<00:00,  9.36it/s]
